9 Лаб. работа

Описание датасета:

Ранг — рейтинг общих продаж.
Имя – название игры.
Платформа - Платформа выпуска игр (т.е. ПК,PS4 и т.д.)
Год - Год выхода игры.
Жанр - Жанр игры.
Издатель - Издатель игры.
NA_Sales — Продажи в Северной Америке (в миллионах)
EU_Sales — Продажи в Европе (в миллионах)
JP_Sales — Продажи в Японии (в миллионах)
Other_Sales — Продажи в остальном мире (в миллионах)
Global_Sales — общий объем продаж по всему миру.

In [289]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as ps
import plotly.graph_objects as go
from apyori import apriori #импорт библиотеки

df = pd.read_csv("vgsales.csv", sep=",")
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


APRIORI - поиск частотных множеств элементов в наборе данных (алгоритм работает на основе ассоциативных правил).

In [290]:
df = df.astype(str)
df.drop(['Rank', 'Publisher'], axis=1,inplace=True)
apry = df.values.tolist()
df.head()

,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.0
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.0,31.37


In [291]:
rules = apriori(transactions = apry, min_support = 0.005, min_confidence = 0.02, min_lift = 1.5, mint_length = 2, max_length = 2)
res = list(rules)
print(res)

[RelationRecord(items=frozenset({'GC', '0.01'}), support=0.01723099168574527, ordered_statistics=[OrderedStatistic(items_base=frozenset({'0.01'}), items_add=frozenset({'GC'}), confidence=0.050971306362502224, lift=1.521621839936712), OrderedStatistic(items_base=frozenset({'GC'}), items_add=frozenset({'0.01'}), confidence=0.5143884892086331, lift=1.5216218399367123)]), RelationRecord(items=frozenset({'0.01', 'PC'}), support=0.030063863116038078, ordered_statistics=[OrderedStatistic(items_base=frozenset({'0.01'}), items_add=frozenset({'PC'}), confidence=0.08893245410800214, lift=1.5376050763381455), OrderedStatistic(items_base=frozenset({'PC'}), items_add=frozenset({'0.01'}), confidence=0.5197916666666667, lift=1.5376050763381452)]), RelationRecord(items=frozenset({'XB', '0.01'}), support=0.02656946620074708, ordered_statistics=[OrderedStatistic(items_base=frozenset({'0.01'}), items_add=frozenset({'XB'}), confidence=0.07859561575476742, lift=1.5831675124971234), OrderedStatistic(items_ba

Support(поддержка) - показатель "частотности"
Lift(лифт)- отношение зависимостей items к независимости (насколько зависят друг от друга)
Confidende(умереннось) - частота срабатывания правила для всего датасета

Функция для вывода

In [292]:
def insp(res):
    lhs =[tuple(ress[2][0][0])[0] for ress in res]
    rhs =[tuple(ress[2][0][1])[0] for ress in res]
    support =[ress[1] for ress in res]
    confidence = [ress[2][0][2] for ress in res]
    lift = [ress[2][0][3] for ress in res]
    return list (zip(lhs,rhs,support,confidence,lift))
headers = ['Left hand side', 'Right hand side', 'Support', 'Confidence', 'Lift']
resulf_df = pd.DataFrame(insp(res), columns=headers)
resulf_df = resulf_df.nlargest(n=10, columns='Lift') #сортировка по столбцу Lift
top_10 = resulf_df[:10]

Вывод сортированного DF

In [293]:
resulf_df

,Left hand side,Right hand side,Support,Confidence,Lift
9,1999.0,N64,0.006145,0.301775,15.701768
81,2016.0,PS4,0.006447,0.311047,15.365327
78,2015.0,PS4,0.008254,0.223127,11.022210
8,1998.0,PS,0.014942,0.654354,9.081071
7,1997.0,PS,0.011327,0.650519,9.027855
6,1996.0,PS,0.009881,0.623574,8.653916
10,1999.0,PS,0.012050,0.591716,8.211791
79,2015.0,PSV,0.006808,0.184039,7.396322
74,2014.0,PSV,0.006145,0.175258,7.043409
11,2000.0,PS,0.009579,0.455587,6.322608


Диаграмма

In [294]:
fig = ps.scatter(top_10, x = 'Support', y = 'Confidence', color = 'Lift')
fig.show()

fig2 = ps.scatter(top_10, x = 'Left hand side', y = 'Right hand side', color = 'Lift', size = 'Support')
fig2.show()